# Тестовое задание

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string

In [ ]:
count_vect = CountVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()
clf = LogisticRegression(solver='liblinear')

## Cкачиваем файл с данными

In [ ]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [ ]:
#df = pd.read_json ('reviews_Electronics_5.json', lines=True )
#df.to_csv ('Electronics_5.csv', index = None)

# Предобработка

In [ ]:
dat = pd.read_csv("Electronics_5.csv")[["asin","reviewText", "overall"]]

## Добавляем бинрную оценку

In [ ]:
dat["bin"] = dat.overall > 3

## Отрежем кусок даты

In [ ]:
dat_t, dat = dat, dat[0:50001]

In [ ]:
train_dat, test_dat, train_ans, test_ans = train_test_split(
    dat.reviewText, dat.bin, test_size=0.2)

## Натравим на необработанные данные

In [ ]:
count_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
tfidf_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
count_train_mtrx_dft = count_vect.transform(train_dat.apply(lambda x: np.str_(x)))
tfidf_train_mtrx_dft = tfidf_vect.transform(train_dat.apply(lambda x: np.str_(x)))
count_test_mtrx_dft = count_vect.transform(test_dat.apply(lambda x: np.str_(x)))
tfidf_test_mtrx_dft = tfidf_vect.transform(test_dat.apply(lambda x: np.str_(x)))

## Приводим к базовой форме

In [ ]:
train_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), train_dat.apply(lambda x: np.str_(x)))))
test_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), test_dat.apply(lambda x: np.str_(x)))))
dat["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), dat.reviewText.apply(lambda x: np.str_(x)))))

## Можно воспользоваться анализатором, который предоставляют векторайзеры

In [ ]:
# anlz = count_vect.build_analyzer()
# dat.words = list(map(lambda s: (" ").join(anlz(s)), dat.reviewText))

## Натравим на обработанные данные

In [ ]:
count_vect.fit(dat.words)
tfidf_vect.fit(dat.words)
count_train_mtrx = count_vect.transform(train_dat_tr)
tfidf_train_mtrx = tfidf_vect.transform(train_dat_tr)
count_test_mtrx = count_vect.transform(test_dat_tr)
tfidf_test_mtrx = tfidf_vect.transform(test_dat_tr)

## Проверим, что получилось

In [ ]:
dat

## Разобьем вектора на выборки

In [ ]:
# count_train_mtrx, count_test_mtrx, count_train_ans, count_test_ans = train_test_split(
#     dat_count, dat.bin, test_size=0.2)
# #dat_count.toarray()

In [ ]:
# tfidf_train_mtrx, tfidf_test_mtrx, tfidf_train_ans, tfidf_test_ans = train_test_split(
#     dat_tfidf, dat.bin, test_size=0.2)
# #dat_tfidf.toarray()

In [ ]:
# count_train_mtrx_dft, count_test_mtrx_dft, count_train_ans_dft, count_test_ans_dft = train_test_split(
#     dat_count_deft, dat.bin, test_size=0.2)

In [ ]:
# tfidf_train_mtrx_dft, tfidf_test_mtrx_dft, tfidf_train_ans_dft, tfidf_test_ans_dft = train_test_split(
#     dat_tfidf_deft, dat.bin, test_size=0.2)

# Обучим модели и посмотрим их точность

## Обработанные

In [ ]:
clf.fit(count_train_mtrx, train_ans)
cout_pred = clf.predict(count_test_mtrx)
accuracy_score(cout_pred, test_ans)

In [ ]:
clf.fit(tfidf_train_mtrx, train_ans)
tfidf_pred = clf.predict(tfidf_test_mtrx)
accuracy_score(tfidf_pred, test_ans)

## Дефолтные

In [ ]:
clf.fit(count_train_mtrx_dft, train_ans)
cout_pred = clf.predict(count_test_mtrx_dft)
accuracy_score(cout_pred, test_ans)

In [ ]:
clf.fit(tfidf_train_mtrx_dft, train_ans)
tfidf_pred = clf.predict(tfidf_test_mtrx_dft)
accuracy_score(tfidf_pred, test_ans)

## Обработанные с оптимальными параметрами

In [ ]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx, train_ans)
print(search.best_params_)

In [ ]:
accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx))

In [ ]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx,  train_ans)
print(search.best_params_)

In [ ]:
accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx))

## Дефолтные с оптимальными параметрами

In [ ]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx_dft, train_ans)
print(search.best_params_)

In [ ]:
accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx_dft))

In [ ]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx_dft, train_ans)
print(search.best_params_)

In [ ]:
accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx_dft))

## Проверим баланс классов

In [ ]:
print("All :\n1 - ", sum(dat.bin), "\n0 - ", len(dat.bin) - sum(dat.bin)) 
print("Train :\n1 - ", sum(train_ans), "\n0 - ", len(train_ans) - sum(train_ans)) 
print("Test :\n1 - ", sum(test_ans), "\n0 - ", len(test_ans) - sum(test_ans)) 